In [1]:
# 필요한 전체 코드 다시 실행해서 세 파일 만들기

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [3]:
# 1. 파일 경로
file_product = "예주나라상품정보.xlsx"
file_category = "표준카테고리카드.xlsx"

# 2. 파일 불러오기
product_df = pd.read_excel(file_product)
category_df = pd.read_excel(file_category)

# 3. 표준카테고리 통합 텍스트
category_df['통합분류'] = category_df[['3차분류', '4차분류', '5차분류']].fillna('').agg(' '.join, axis=1)

# 4. 예주나라 상품 검색문장 생성
product_df['검색문장'] = product_df[['카테고리명', '품목명', '이전품목명']].fillna('').agg(' '.join, axis=1)
product_texts = product_df[['카테고리명', '검색문장']].dropna(subset=['카테고리명']).drop_duplicates().reset_index(drop=True)

# 5. 벡터화 및 유사도
all_texts = pd.concat([product_texts['검색문장'], category_df['통합분류']], ignore_index=True)
vectorizer = TfidfVectorizer().fit(all_texts)
product_vecs = vectorizer.transform(product_texts['검색문장'])
category_vecs = vectorizer.transform(category_df['통합분류'])
similarity_matrix = cosine_similarity(product_vecs, category_vecs)

# 6. 추천 생성 함수
def make_recommendations(top_k):
    results = []
    for i, row in product_texts.iterrows():
        category_name = row['카테고리명']
        original_text = row['검색문장']
        top_idx = similarity_matrix[i].argsort()[-top_k:][::-1]
        for rank, j in enumerate(top_idx):
            cat_row = category_df.iloc[j]
            results.append({
                '카테고리명': category_name,
                '검색문장': original_text,
                '추천순위': rank + 1,
                '추천카테고리코드': cat_row['카테고리코드'],
                '1차분류': cat_row['1차분류'],
                '2차분류': cat_row['2차분류'],
                '3차분류': cat_row['3차분류'],
                '4차분류': cat_row['4차분류'],
                '5차분류': cat_row['5차분류'],
                '유사도점수': round(similarity_matrix[i][j], 4)
            })
    return pd.DataFrame(results)

# 7. Top 3, Top 1 추천
top3_df = make_recommendations(top_k=3)
top1_df = make_recommendations(top_k=1)

# 8. 예주나라 상품 기본 정보 병합
product_info = product_df[['카테고리명', '품목명', '이전품목명']].dropna(subset=['카테고리명']).drop_duplicates()
top3_df = pd.merge(top3_df, product_info, on='카테고리명', how='left')
top1_df = pd.merge(top1_df, product_info, on='카테고리명', how='left')

# 9. Top1 추천을 예주나라 상품정보에 병합
top1_clean = top1_df.drop_duplicates(subset=['카테고리명'])[
    ['카테고리명', '추천카테고리코드', '1차분류', '2차분류', '3차분류', '4차분류', '5차분류', '유사도점수']
]
merged_product_df = pd.merge(product_df, top1_clean, on='카테고리명', how='left')

# 10. 결과 저장
top3_df.to_excel("카테고리_유사도_TOP3추천.xlsx", index=False)
top1_df.to_excel("카테고리_유사도_1순위자동매핑.xlsx", index=False)
merged_product_df.to_excel("예주나라상품정보_추천카테고리_추가.xlsx", index=False)

"/mnt/data 경로에 3개 파일 저장 완료!"


'/mnt/data 경로에 3개 파일 저장 완료!'

In [ ]:
# top3_df, top1_df, merged_product_df에 '품목코드', '품목명' 컬럼 추가

# 예주나라 상품에서 품목코드도 포함한 정보 추출
product_info_full = product_df[['카테고리명', '품목코드', '품목명', '이전품목명']].dropna(subset=['카테고리명']).drop_duplicates()

# top3, top1에 병합
top3_df = pd.merge(top3_df, product_info_full, on=['카테고리명', '품목명', '이전품목명'], how='left')
top1_df = pd.merge(top1_df, product_info_full, on=['카테고리명', '품목명', '이전품목명'], how='left')

# 순서 정리: 품목코드, 품목명, 카테고리 추천 관련 항목 앞으로
cols_order = ['품목코드', '품목명', '카테고리명', '추천순위', '추천카테고리코드', '1차분류', '2차분류', '3차분류', '4차분류', '5차분류', '유사도점수', '검색문장', '이전품목명']
top3_df = top3_df[[col for col in cols_order if col in top3_df.columns]]
top1_df = top1_df[[col for col in cols_order if col in top1_df.columns]]

# 병합된 product_df도 순서만 정리 (이미 품목코드, 품목명 있음)
merged_product_df = merged_product_df[['품목코드', '품목명', '카테고리명'] + 
                                       [col for col in merged_product_df.columns if col.startswith('추천') or col.endswith('분류') or col == '유사도점수'] + 
                                       [col for col in merged_product_df.columns if col not in ['품목코드', '품목명', '카테고리명', '추천카테고리코드', '1차분류', '2차분류', '3차분류', '4차분류', '5차분류', '유사도점수']]]

# 결과 다시 저장
top3_df.to_excel("카테고리_유사도_TOP3추천.xlsx", index=False)
top1_df.to_excel("카테고리_유사도_1순위자동매핑.xlsx", index=False)
merged_product_df.to_excel("예주나라상품정보_추천카테고리_추가.xlsx", index=False)

"파일 3개에 품목코드/품목명 포함해서 다시 저장 완료!"

'파일 3개에 품목코드/품목명 포함해서 다시 저장 완료!'

: 